In [14]:
# Dependencies
import numpy as np
import pandas as pd
import requests
from census import Census

# Census API Keys
from config import (api_key)
c = Census(api_key, year=2017)

### The poverty numbers in the dataframe is "Number of persons whose income in the past 12 months is below the poverty level"

In [15]:
#"B25077_001E": "Median Home Value"

census_data = c.acs5.get(("NAME",
                          "B01003_001E",
                          "B17001_002E"), 
                         {'for': 'county:*','in':'state:19'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Total Poverty Rate (%)"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)


# Final DataFrame
census_pd = census_pd[["Name", "Population",
                       "Poverty Count", "Total Poverty Rate (%)"]]


split = census_pd["Name"].str.split(",", n = 1, expand = True)
census_pd["County"]= split[0]
census_pd["State"]=split[1]
census_pd = census_pd[["State", "County", "Population", 
                       "Poverty Count", "Total Poverty Rate (%)"]]

census_pd = census_pd.set_index("State")
census_pd.to_csv("2017_Poverty_Data.csv")
census_pd.head()

,County,Population,Poverty Count,Total Poverty Rate (%)
State,,,,
Iowa,Delaware County,17326.0,1570.0,9.061526
Iowa,Jefferson County,17945.0,2510.0,13.987183
Iowa,Wayne County,6405.0,1001.0,15.628415
Iowa,Allamakee County,13940.0,1550.0,11.119082
Iowa,Linn County,220008.0,20280.0,9.217847
